# U-Net Model Training for Fault Segmentation (keras)

## Step 1: Import Libraries

First, we import all the necessary libraries. This includes standard libraries like `os` and `numpy`, deep learning libraries from `tensorflow.keras`, and the custom modules `DataGenerator` and `unet` from our project.

In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

# TensorFlow Keras imports
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras import backend as K # type: ignore
from tensorflow.keras.utils import Sequence, get_custom_objects # type: ignore

from unet3_keras import cross_entropy_balanced
get_custom_objects()['cross_entropy_balanced'] = cross_entropy_balanced

# Set random seeds for reproducibility, as in the original script
np.random.seed(12345)
tf.random.set_seed(1234)

In [ ]:
try:
    ROOT_DIR = os.path.dirname(os.path.abspath(__file__))  # for .py scripts
except NameError:
    ROOT_DIR = os.getcwd()  # for Jupyter notebooks

## Step 1a: Verify GPU Availability
This cell checks if TensorFlow can detect a GPU. If a GPU is available, it enables memory growth to prevent TensorFlow from allocating all of the GPU's memory at once.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        
        # GPU Warm-up to ensure it's ready
        print("Warming up GPU...")
        with tf.device('/GPU:0'):
            dummy_tensor = tf.random.normal([1, 1])
            _ = dummy_tensor * 2.0
        print("GPU is warmed up and ready.")

    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU detected. Running on CPU.")

## Step 2: Define Paths and Parameters

In [ ]:
# --- Input: Original .dat directories ---
base_dir = os.path.abspath(os.path.join(ROOT_DIR, "..", "data", "data_from_Wu"))
train_seis_dir = os.path.join(base_dir, "train/seis")
train_fault_dir = os.path.join(base_dir, "train/fault")
validation_seis_dir = os.path.join(base_dir, "validation/seis")
validation_fault_dir = os.path.join(base_dir, "validation/fault")
prediction_seis_dir = os.path.join(base_dir, "prediction/f3d")

# --- Output: Processed .npy directories ---
processed_data_dir = os.path.join(ROOT_DIR, "data")
train_dir_new = os.path.join(processed_data_dir, 'train_npy')
validation_dir_new = os.path.join(processed_data_dir, 'validation_npy')
prediction_dir_new = os.path.join(processed_data_dir, 'prediction_npy')

# --- Model/Log dirs (unchanged) ---
model_dir = os.path.join(ROOT_DIR, "model")
log_dir = os.path.join(ROOT_DIR, "output", "logs")
checkpoint_dir = os.path.join(ROOT_DIR, "output", "checkpoints")

In [ ]:
# --- Training Parameters (from train_keras.py) ---
params = {
    'batch_size': 1,
    'dim': (128, 128, 128),
    'n_channels': 1,
    'shuffle': True
}


In [ ]:
# Training configuration
epochs = 25
learning_rate = 1e-4

## Step 3: Prepare Data

This step involves converting the original `.dat` files into the `.npy` format that our `DataGenerator` expects. We also split the data into training and validation sets. This is a crucial preprocessing step.

In [ ]:
def convert_dat_to_npy():
    """Creates directories and converts .dat files to .npy format."""

    # Clean up old processed data folders
    for folder in [train_dir_new, validation_dir_new, prediction_dir_new]:
        if os.path.exists(folder):
            shutil.rmtree(folder)
            print(f"Removed old directory: {folder}")

    # Create new directory structure
    os.makedirs(os.path.join(train_dir_new, 'seis'), exist_ok=True)
    os.makedirs(os.path.join(train_dir_new, 'fault'), exist_ok=True)
    os.makedirs(os.path.join(validation_dir_new, 'seis'), exist_ok=True)
    os.makedirs(os.path.join(validation_dir_new, 'fault'), exist_ok=True)
    os.makedirs(os.path.join(prediction_dir_new, 'seis'), exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("Created new directory structure for processed data and model artifacts.")

    def process_files(file_list, source_seis, source_fault, dest_dir):
        """Reads .dat, reshapes, and saves as .npy."""
        for filename in file_list:
            # Process seismic data
            seis_dat_path = os.path.join(source_seis, filename)
            seis_npy_path = os.path.join(dest_dir, 'seis', filename.replace('.dat', '.npy'))
            seismic_data = np.fromfile(seis_dat_path, dtype=np.single)

            if seismic_data.size == np.prod(params['dim']):
                # Normal case (1 volume)
                seismic_data = seismic_data.reshape(params['dim'])
                np.save(seis_npy_path, seismic_data)
            elif seismic_data.size % np.prod(params['dim']) == 0:
                # Special case: prediction files contain multiple volumes
                num_volumes = seismic_data.size // np.prod(params['dim'])
                seismic_data = seismic_data.reshape((num_volumes, *params['dim']))
                for i in range(num_volumes):
                    sub_volume = seismic_data[i]
                    volume_name = filename.replace('.dat', f'_vol{i+1}.npy')
                    np.save(os.path.join(dest_dir, 'seis', volume_name), sub_volume)
                print(f"🔁 Split {filename} into {num_volumes} volumes.")
                continue
            else:
                raise ValueError(f"❌ Unexpected shape for file {filename} with {seismic_data.size} elements.")

            # Process fault data if available
            fault_dat_path = os.path.join(source_fault, filename)
            fault_npy_path = os.path.join(dest_dir, 'fault', filename.replace('.dat', '.npy'))
            if os.path.exists(fault_dat_path):
                fault_data = np.fromfile(fault_dat_path, dtype=np.single).reshape(params['dim'])
                np.save(fault_npy_path, fault_data)
        print(f"Processed {len(file_list)} files for {os.path.basename(dest_dir)}.")

    # Get file lists
    train_dat_files = [f for f in os.listdir(train_seis_dir) if f.endswith('.dat')]
    validation_dat_files = [f for f in os.listdir(validation_seis_dir) if f.endswith('.dat')]

    if os.path.exists(prediction_seis_dir):
        prediction_dat_files = [f for f in os.listdir(prediction_seis_dir) if f.endswith('.dat')]
    else:
        prediction_dat_files = []
        print(f"⚠️ Warning: Prediction folder '{prediction_seis_dir}' not found. Skipping prediction data.")

    print("\nStarting data conversion...")
    process_files(train_dat_files, train_seis_dir, train_fault_dir, train_dir_new)
    process_files(validation_dat_files, validation_seis_dir, validation_fault_dir, validation_dir_new)
    process_files(prediction_dat_files, prediction_seis_dir, prediction_seis_dir, prediction_dir_new)
    print("\n✅ Data preparation complete. All .dat files converted to .npy.")


In [ ]:
# Run the conversion
convert_dat_to_npy()

## Step 4: Create Data Generators

Now that our data is in the correct format and location, we can use the `DataGenerator` from `utils.py` to feed it to our model efficiently. We create one generator for the training set and one for the validation set.

In [ ]:
class DataGenerator(Sequence):
    'Generates data for a Keras model'
    def __init__(self, dpath, fpath, data_IDs, batch_size, dim, n_channels, shuffle):
        'Initialization'
        super().__init__()
        self.dim = dim
        self.dpath = dpath
        self.fpath = fpath
        self.batch_size = batch_size
        self.data_IDs = data_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generates one batch of data'
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        data_IDs_temp = [self.data_IDs[k] for k in indexes]
        X, Y = self.__data_generation(data_IDs_temp)
        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, data_IDs_temp):
        'Generates data containing batch_size samples with augmentation'
        # The augmentation doubles the batch size
        effective_batch_size = len(data_IDs_temp) * 2
        X = np.empty((effective_batch_size, *self.dim, self.n_channels), dtype=np.single)
        Y = np.empty((effective_batch_size, *self.dim, self.n_channels), dtype=np.single)
        
        for i, ID in enumerate(data_IDs_temp):
            # Load pre-converted .npy files
            gx = np.load(os.path.join(self.dpath, ID + '.npy'))
            fx = np.load(os.path.join(self.fpath, ID + '.npy'))
            
            # Normalize seismic data
            xm = np.mean(gx)
            xs = np.std(gx)
            gx = (gx - xm) / (xs + 1e-8) # Add epsilon to avoid division by zero
            
            # Transpose as in original script
            gx = np.transpose(gx)
            fx = np.transpose(fx)

            gx_reshaped = np.reshape(gx, (*self.dim, self.n_channels))
            fx_reshaped = np.reshape(fx, (*self.dim, self.n_channels))
            
            # Original and flipped data for augmentation
            X[i * 2, ] = gx_reshaped
            Y[i * 2, ] = fx_reshaped
            X[i * 2 + 1, ] = np.flipud(gx_reshaped)
            Y[i * 2 + 1, ] = np.flipud(fx_reshaped)

        return X, Y

# --- Instantiate the Generators ---
seismPathT = os.path.join(train_dir_new, "seis/")
faultPathT = os.path.join(train_dir_new, "fault/")
seismPathV = os.path.join(validation_dir_new, "seis/")
faultPathV = os.path.join(validation_dir_new, "fault/")

train_ID = [os.path.splitext(f)[0] for f in os.listdir(seismPathT)]
valid_ID = [os.path.splitext(f)[0] for f in os.listdir(seismPathV)]

train_generator = DataGenerator(dpath=seismPathT, fpath=faultPathT, data_IDs=train_ID, **params)
valid_generator = DataGenerator(dpath=seismPathV, fpath=faultPathV, data_IDs=valid_ID, **params)

print("Data generators for training and validation have been created.")

In [ ]:
print(f"Number of training pairs: {len(train_ID)}")
print(f"Number of validation pairs: {len(valid_ID)}")

## Step 5: Build and Compile the Model

We will now use the `unet` function from `unet3.py` to build our model architecture. After building it, we compile it with an optimizer, a loss function, and metrics to monitor.

In [ ]:
def unet(input_size=(128, 128, 128, 1)):
    """Defines the 3D U-Net model architecture from unet3_keras.py."""
    inputs = Input(input_size)
    
    # Encoder Path
    conv1 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    # Bottleneck
    conv4 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv4)

    # Decoder Path
    up5 = concatenate([UpSampling3D(size=(2, 2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling3D(size=(2, 2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling3D(size=(2, 2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv7)

    # Output Layer
    conv8 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=[inputs], outputs=[conv8])
    return model

def _to_tensor(x, dtype):
    """Convert the input `x` to a tensor of type `dtype`."""
    return tf.convert_to_tensor(x, dtype=dtype)

def cross_entropy_balanced(y_true, y_pred):
    """Custom balanced cross-entropy loss function from unet3_keras.py."""
    _epsilon = _to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred = tf.clip_by_value(y_pred, _epsilon, 1 - _epsilon)
    y_pred = tf.math.log(y_pred / (1 - y_pred))

    y_true = tf.cast(y_true, tf.float32)
    count_neg = tf.reduce_sum(1. - y_true)
    count_pos = tf.reduce_sum(y_true)
    beta = count_neg / (count_neg + count_pos)
    pos_weight = beta / (1 - beta)

    cost = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, labels=y_true, pos_weight=pos_weight)
    cost = tf.reduce_mean(cost * (1 - beta))
    
    return tf.where(tf.equal(count_pos, 0.0), 0.0, cost)

# Build and compile the model
model = unet(input_size=(*params['dim'], params['n_channels']))
model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss=cross_entropy_balanced,
              metrics=['accuracy'])

model.summary()

## Step 6: Define Callbacks

In [ ]:
filepath = os.path.join(checkpoint_dir, "fseg-{epoch:02d}.keras") # <-- FIXED
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')

# TensorBoard callback for logging
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# List of callbacks to pass to the model
callbacks_list = [checkpoint, tensorboard_callback]

print("Callbacks defined. Checkpoints will be saved in the new .keras format.")
print("Starting model training...")

## Step 7: Train the Model

With everything set up, we can now start training the model using the `fit` method. The training and validation data will be provided through our generators.

In [ ]:
history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    verbose=1)

print("\nModel training has been completed.")

## Step 8: Save the Final Model

After training is complete, we save the final model to the `model` directory for future use.

In [ ]:
# Create a unique name for the final model using the .hdf5 extension
now = datetime.now()
date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
num_pairs = len(train_ID)
model_name = f"unet_keras_model_{num_pairs}pairs_{epochs}epochs_{date_time}.keras" # <-- FIXED
model_path = os.path.join(model_dir, model_name)

# Save the model
model.save(model_path)

print(f"The final model has been saved to: {model_path}")

## Step 9: Visualize Training History

Finally, we visualize the training and validation accuracy and loss over the epochs. This helps us understand how well the model has learned.

In [ ]:
def show_history(history, save_path=None):
    """
    Plots the training and validation accuracy and loss from a Keras history object.
    """
    print("Plotting training history...")
    
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

    # --- Plot Accuracy ---
    ax1.plot(history.history['accuracy'], label='Training Accuracy', color='b', marker='o', linestyle='-')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', color='r', marker='o', linestyle='--')
    ax1.set_title('Model Accuracy', fontsize=18)
    ax1.set_ylabel('Accuracy', fontsize=14)
    ax1.set_xlabel('Epoch', fontsize=14)
    ax1.legend(loc='lower right', fontsize=12)
    ax1.grid(True)

    # --- Plot Loss ---
    ax2.plot(history.history['loss'], label='Training Loss', color='b', marker='o', linestyle='-')
    ax2.plot(history.history['val_loss'], label='Validation Loss', color='r', marker='o', linestyle='--')
    ax2.set_title('Model Loss', fontsize=18)
    ax2.set_ylabel('Loss', fontsize=14)
    ax2.set_xlabel('Epoch', fontsize=14)
    ax2.legend(loc='upper right', fontsize=12)
    ax2.grid(True)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        print(f"History plot saved to: {save_path}")
        
    plt.show()

In [ ]:
# Use os.path.splitext to robustly get the model name without its extension
base_name, _ = os.path.splitext(model_name)
plot_name = f"{base_name}_history.png"
plot_path = os.path.join(ROOT_DIR, "output", "history_plots", plot_name)

# Ensure history_plots directory exists
os.makedirs(os.path.dirname(plot_path), exist_ok=True)

# Call the function to save plot
show_history(history, save_path=plot_path)